In [ ]:
# Libraries 
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
# import sys
# import os
# sys.path.append(os.path.abspath(os.path.join('..', 'src'))) 


# Configuration
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)


### Phase 1: Exploration and Cleanup

In [ ]:
# Exploratory Data Analysis
    # create dataframes DONE
    # dataframe level 
        # visual exploration with .head(), .tail() and sample() DONE
        # dataframe overview with .info() DONE
        # check duplicates with .duplicated().sum DONE
            # create separate dataframe with duplicates if needed DONE
        # check null values .isnull().sum() DONE
        # explore main statistical metrics for numerical and non numerical columns with .describe() and .describe(include = "object")
    # column level
        # review columns names with .columns
        # review unique values with .unique()
        # review value count with .values_count()
        # check null values with .isnull().sum()
        # atypical values
    # create function/s for the EDA
# Transformation


In [ ]:
# open data

df_customer_flights = pd.read_csv("data/customer_flight_activity.csv")
df_customer_loyalty = pd.read_csv("data/customer_loyalty_history.csv")
dataframes = {"df_customer_flight" : df_customer_flights, "df_customer_loyalty" : df_customer_loyalty}

In [68]:

def explore_dataframes (df_dict):
    """
    Provides relevant information for a Exploratory Data Analysis.
    
    This function receives a dict with dataframes and executes a series of 
    functions aiming to provide relevant information to analize the data.

    Args:

    Returns:
    
    """
    duplicate_dataframes_dict = {}
    for name, df in df_dict.items():
        print(f" \n----------- DATAFRAME NAME: '{name.upper().replace('_',' ')}' -----------\n")
        print(f"\nDataframe INFO - '{name.upper().replace('_',' ')}':\n")
        display(df.info())
        print(f"\nFirst 5 rows of the DataFrame - {name.upper().replace('_',' ')}:")
        display(df.head())
        print(f"\nLast 5 rows of the DataFrame - {name.upper().replace('_',' ')}:")
        display(df.tail())
        print(f"\nSample rows of the DataFrame - {name.upper().replace('_',' ')}:")
        display(df.sample(5))
        print(f" \nDUPLICATES COUNT in '{name.upper().replace('_',' ')}'is: {df.duplicated().sum()}\n")
        if df.duplicated().sum() > 0:
            duplicates_df_name = name + "_duplicates"
            duplicates_df = df[df.duplicated(keep=False)]
            print("The following dataframe contains ALL the duplicate rows:\n")
            display(duplicates_df)
            duplicate_dataframes_dict[duplicates_df_name] = duplicates_df
        print(f" \nCount of rows with ALL null values in '{name.upper().replace('_',' ')}'is: {df.isnull().all(axis=1).sum()}\n")
        print(f" \nCount of columns with ALL null values in '{name.upper().replace('_',' ')}'is: {df.isnull().all().sum()}\n")
        
        return duplicate_dataframes_dict
        
        
      



In [70]:
duplicate_dataframes_dict = explore_dataframes(dataframes)

 
----------- DATAFRAME NAME: 'DF CUSTOMER FLIGHT' -----------


Dataframe INFO - 'DF CUSTOMER FLIGHT':

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405624 entries, 0 to 405623
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Loyalty Number               405624 non-null  int64  
 1   Year                         405624 non-null  int64  
 2   Month                        405624 non-null  int64  
 3   Flights Booked               405624 non-null  int64  
 4   Flights with Companions      405624 non-null  int64  
 5   Total Flights                405624 non-null  int64  
 6   Distance                     405624 non-null  int64  
 7   Points Accumulated           405624 non-null  float64
 8   Points Redeemed              405624 non-null  int64  
 9   Dollar Cost Points Redeemed  405624 non-null  int64  
dtypes: float64(1), int64(9)
memory usage: 30.9 MB


None


First 5 rows of the DataFrame - DF CUSTOMER FLIGHT:


,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0
3,100214,2017,1,0,0,0,0,0.0,0,0
4,100272,2017,1,0,0,0,0,0.0,0,0



Last 5 rows of the DataFrame - DF CUSTOMER FLIGHT:


...


Sample rows of the DataFrame - DF CUSTOMER FLIGHT:


,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
323865,247721,2018,8,0,0,0,0,0.0,0,0
261169,723968,2018,7,2,2,4,2192,219.0,698,56
116515,904317,2017,7,0,0,0,0,0.0,0,0
74536,470381,2017,5,13,4,17,2193,219.0,0,0
392150,283133,2018,12,0,0,0,0,0.0,0,0


 
DUPLICATES COUNT in 'DF CUSTOMER FLIGHT'is: 1864

The following dataframe contains ALL the duplicate rows:



,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
41,101902,2017,1,0,0,0,0,0.0,0,0
42,101902,2017,1,0,0,0,0,0.0,0,0
226,112142,2017,1,0,0,0,0,0.0,0,0
227,112142,2017,1,0,0,0,0,0.0,0,0
477,126100,2017,1,0,0,0,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
405111,971370,2018,12,0,0,0,0,0.0,0,0
405409,988392,2018,12,0,0,0,0,0.0,0,0
405410,988392,2018,12,0,0,0,0,0.0,0,0
405436,989528,2018,12,0,0,0,0,0.0,0,0


 
Count of rows with ALL null values in 'DF CUSTOMER FLIGHT'is: 0

 
Count of columns with ALL null values in 'DF CUSTOMER FLIGHT'is: <bound method NDFrame._add_numeric_operations.<locals>.sum of Loyalty Number                 False
Year                           False
Month                          False
Flights Booked                 False
Flights with Companions        False
Total Flights                  False
Distance                       False
Points Accumulated             False
Points Redeemed                False
Dollar Cost Points Redeemed    False
dtype: bool>

